In [24]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
import sklearn as sk
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV

# Part 1

Consider four possible models for predicting house prices:

Using only the size and number of rooms.
Using size, number of rooms, and building type.
Using size and building type, and their interaction.
Using a 5-degree polynomial on size, a 5-degree polynomial on number of rooms, and also building type.

Set up a pipeline for each of these four models.

Then, get predictions on the test set for each of your pipelines, and compute the root mean squared error. Which model performed best?

Note: You should only use the function train_test_split() one time in your code; that is, we should be predicting on the same test set for all three models.

In [2]:
data = pd.read_csv("AmesHousing.csv")
data.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,...,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112.0,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.0,Unf,0.0,441.0,1080.0,...,Y,SBrkr,1656,0,0,1656,1.0,0.0,1,0,3,1,TA,7,Typ,2,Gd,Attchd,1960.0,Fin,2.0,528.0,TA,TA,P,210,62,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,...,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,...,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,NaN,0.0,Gd,TA,CBlock,TA,TA,No,ALQ,1065.0,Unf,0.0,1045.0,2110.0,...,Y,SBrkr,2110,0,0,2110,1.0,0.0,2,1,3,1,Ex,8,Typ,2,TA,Attchd,1968.0,Fin,2.0,522.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,...,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [3]:
X = data.drop('SalePrice', axis=1)
y = data['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [4]:
ct = ColumnTransformer(
    [
        ('standardize', StandardScaler(), ["TotRms AbvGrd", "Gr Liv Area"])
    ],
    remainder = 'drop'
)

lr_pipeline = Pipeline(
    [
        ('preprocessing', ct),
        ('regression', LinearRegression())
    ]
)

model1 = lr_pipeline.fit(X_train, y_train)

y_preds = model1.predict(X_test)

mse1 = mean_squared_error(y_test, y_preds)
rmse1 = np.sqrt(mse1)

r2_mod1 = r2_score(y_test, y_preds)

print('The RMSE for Model 1 is:', rmse1.round(2))
print('The R^2 for Model 1 is:', r2_mod1.round(4))
print('Coefficients:', model1.named_steps['regression'].coef_.round(2))
print('Intercept:', model1.named_steps['regression'].intercept_.round(2))

The RMSE for Model 1 is: 58893.7
The R^2 for Model 1 is: 0.5477
Coefficients: [-13431.36  65495.93]
Intercept: 180348.27
The RMSE for Model 1 is: 58893.7
The R^2 for Model 1 is: 0.5477
Coefficients: [-13431.36  65495.93]
Intercept: 180348.27


In [5]:
ct = ColumnTransformer(
  [  
    ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"]),
    ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"])
  ],
  remainder = "drop"
)


lr_pipeline = Pipeline(
  [("preprocessing", ct),
  ("regression", LinearRegression())]
)

model2 = lr_pipeline.fit(X_train,y_train)

mod2_y_preds = model2.predict(X_test)

mse2 = mean_squared_error(y_test, mod2_y_preds)
rmse2 = np.sqrt(mse2)

r2_mod2 = r2_score(y_test, mod2_y_preds)

print('The RMSE for Model 2 is:', rmse2.round(2))
print('The R^2 for Model 2 is:', r2_mod2.round(4))
print('Coefficients:', model2.named_steps['regression'].coef_.round(2))
print('Intercept:', model2.named_steps['regression'].intercept_.round(2))

The RMSE for Model 2 is: 57301.68
The R^2 for Model 2 is: 0.5718
Coefficients: [ 60559.83  -6258.65  20525.43 -36053.41 -30890.36    352.85  46065.48]
Intercept: 161378.61
The RMSE for Model 2 is: 57301.68
The R^2 for Model 2 is: 0.5718
Coefficients: [ 60559.83  -6258.65  20525.43 -36053.41 -30890.36    352.85  46065.48]
Intercept: 161378.61


In [8]:
ct = ColumnTransformer(
  [  
    ("standardize", StandardScaler(), ["Gr Liv Area"]),
    ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"])
  ],
  remainder = "drop"
).set_output(transform="pandas")

X_train_dummified = ct.fit_transform(X_train)
X_train_dummified

ct_inter = ColumnTransformer(
  [
    ("int", PolynomialFeatures(interaction_only = True), ["standardize__Gr Liv Area", "dummify__Bldg Type_1Fam"]),
    ("int2", PolynomialFeatures(interaction_only = True), ["standardize__Gr Liv Area", "dummify__Bldg Type_2fmCon"]),
    ("int3", PolynomialFeatures(interaction_only = True), ["standardize__Gr Liv Area", "dummify__Bldg Type_Duplex"]),
    ("int4", PolynomialFeatures(interaction_only = True), ["standardize__Gr Liv Area", "dummify__Bldg Type_Twnhs"]),
    ("int5", PolynomialFeatures(interaction_only = True), ["standardize__Gr Liv Area", "dummify__Bldg Type_TwnhsE"])
  ],
  remainder = "drop"
).set_output(transform = "pandas")

lr_pipeline = Pipeline(
  [("preprocessing", ct),
   ("interactions", ct_inter),
  ("regression", LinearRegression())]
)


mod3 = lr_pipeline.fit(X_train, y_train)

mod3_y_preds = mod3.predict(X_test)
mse3 = mean_squared_error(y_test, mod3_y_preds)
rmse3 = np.sqrt(mse3)

r2_mod3 = r2_score(y_test, mod3_y_preds)

print('The RMSE for Model 3 is:', rmse3.round(2))
print('The R^2 for Model 3 is:', r2_mod3.round(4))
print('Coefficients:', mod3.named_steps['regression'].coef_.round(2))
print('Intercept:', mod3.named_steps['regression'].intercept_.round(2))

The RMSE for Model 3 is: 56402.77
The R^2 for Model 3 is: 0.5851
Coefficients: [     0.     8527.38  18240.78  14696.75     -0.     8527.38 -36949.18
 -28166.16     -0.     8527.38 -29660.31 -16454.24     -0.     8527.38
  -5406.02   2148.37     -0.     8527.38  53774.72  36302.67]
Intercept: 163468.31
The RMSE for Model 3 is: 56402.77
The R^2 for Model 3 is: 0.5851
Coefficients: [     0.     8527.38  18240.78  14696.75     -0.     8527.38 -36949.18
 -28166.16     -0.     8527.38 -29660.31 -16454.24     -0.     8527.38
  -5406.02   2148.37     -0.     8527.38  53774.72  36302.67]
Intercept: 163468.31


In [11]:
ct = ColumnTransformer(
  [  
    ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"]),
    ('polynomial', PolynomialFeatures(degree=5, include_bias=False), ["Gr Liv Area", "TotRms AbvGrd"]),
    ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"])
  ],
  remainder = "drop"
)


lr_pipeline = Pipeline(
  [("preprocessing", ct),
   
  ("regression", LinearRegression())]
)


model4 = lr_pipeline.fit(X_train, y_train)

mod4_y_preds = model4.predict(X_test)

mse4 = mean_squared_error(y_test, mod4_y_preds)
rmse4 = np.sqrt(mse4)

r2_mod4 = r2_score(y_test, mod4_y_preds)

print('The RMSE for Model 4 is:', rmse4.round(2))
print('The R^2 for Model 4 is:', r2_mod4.round(4))
print('Coefficients:', model4.named_steps['regression'].coef_.round(2))
print('Intercept:', model4.named_steps['regression'].intercept_.round(2))

The RMSE for Model 4 is: 60132.12
The R^2 for Model 4 is: 0.5284
Coefficients: [ 0.   -0.06 -0.12 -0.    0.22 -1.11 -0.02 -0.    0.02 -7.34 -0.12  0.    0.
 -0.01  1.3  -0.39  0.   -0.    0.   -0.    0.06 -4.29 -0.   -0.    0.   -0.
  0.  ]
Intercept: 45458.25
The RMSE for Model 4 is: 60132.12
The R^2 for Model 4 is: 0.5284
Coefficients: [ 0.   -0.06 -0.12 -0.    0.22 -1.11 -0.02 -0.    0.02 -7.34 -0.12  0.    0.
 -0.01  1.3  -0.39  0.   -0.    0.   -0.    0.06 -4.29 -0.   -0.    0.   -0.
  0.  ]
Intercept: 45458.25


In [13]:
RMSE_table = {"Models": ["Model 1", "Model 2","Model 3","Model 4"],
         "RMSE": [rmse1,rmse2,rmse3,rmse4]}

pd.DataFrame(RMSE_table)

,Models,RMSE
0,Model 1,58893.696752
1,Model 2,57301.682883
2,Model 3,56402.767011
3,Model 4,60132.120649


As we can see from the above table, the model with the lowest RMSE is Model 3.

# Part 2

Once again consider four modeling options for house price:

Using only the size and number of rooms.
Using size, number of rooms, and building type.
Using size and building type, and their interaction.
Using a 5-degree polynomial on size, a 5-degree polynomial on number of rooms, and also building type.

Use cross_val_score with the pipelines you made earlier to find the cross-validated root mean squared error for each model.

Which do you prefer? Does this agree with your conclusion from earlier?

In [16]:
ct = ColumnTransformer(
    [
        ('standardize', StandardScaler(), ["TotRms AbvGrd", "Gr Liv Area"])
    ],
    remainder = 'drop'
)

lr_pipeline = Pipeline(
    [
        ('preprocessing', ct),
        ('regression', LinearRegression())
    ]
)

model1 = lr_pipeline.fit(X_train, y_train)

y_preds = model1.predict(X_test)

CV1 = cross_val_score(lr_pipeline, X_train, y_train, cv=5, scoring="neg_mean_squared_error")
cv_rmse1 = np.sqrt(-CV1)

print('Model 1 CV RMSE:', cv_rmse1.mean().round(2))

CV RMSE: 54922.11
CV RMSE: 54922.11


In [18]:
ct = ColumnTransformer(
  [  
    ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"]),
    ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"])
  ],
  remainder = "drop"
)


lr_pipeline = Pipeline(
  [("preprocessing", ct),
  ("regression", LinearRegression())]
)

model2 = lr_pipeline.fit(X_train,y_train)

CV2 = cross_val_score(lr_pipeline, X_train, y_train, cv=5, scoring="neg_mean_squared_error")
cv_rmse2 = np.sqrt(-CV2)

print('Model 2 CV RMSE:', cv_rmse2.mean().round(2))

Model 2 CV RMSE: 53092.36
Model 2 CV RMSE: 53092.36


In [19]:
ct = ColumnTransformer(
  [  
    ("standardize", StandardScaler(), ["Gr Liv Area"]),
    ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"])
  ],
  remainder = "drop"
).set_output(transform="pandas")

X_train_dummified = ct.fit_transform(X_train)
X_train_dummified

ct_inter = ColumnTransformer(
  [
    ("int", PolynomialFeatures(interaction_only = True), ["standardize__Gr Liv Area", "dummify__Bldg Type_1Fam"]),
    ("int2", PolynomialFeatures(interaction_only = True), ["standardize__Gr Liv Area", "dummify__Bldg Type_2fmCon"]),
    ("int3", PolynomialFeatures(interaction_only = True), ["standardize__Gr Liv Area", "dummify__Bldg Type_Duplex"]),
    ("int4", PolynomialFeatures(interaction_only = True), ["standardize__Gr Liv Area", "dummify__Bldg Type_Twnhs"]),
    ("int5", PolynomialFeatures(interaction_only = True), ["standardize__Gr Liv Area", "dummify__Bldg Type_TwnhsE"])
  ],
  remainder = "drop"
).set_output(transform = "pandas")

lr_pipeline = Pipeline(
  [("preprocessing", ct),
   ("interactions", ct_inter),
  ("regression", LinearRegression())]
)


mod3 = lr_pipeline.fit(X_train, y_train)

CV3 = cross_val_score(lr_pipeline, X_train, y_train, cv=5, scoring="neg_mean_squared_error")
cv_rmse3 = np.sqrt(-CV3)

print('Model 3 CV RMSE:', cv_rmse3.mean().round(2))

Model 3 CV RMSE: 52561.8
Model 3 CV RMSE: 52561.8


In [20]:
ct = ColumnTransformer(
  [  
    ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"]),
    ('polynomial', PolynomialFeatures(degree=5, include_bias=False), ["Gr Liv Area", "TotRms AbvGrd"]),
    ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"])
  ],
  remainder = "drop"
)


lr_pipeline = Pipeline(
  [("preprocessing", ct),
   
  ("regression", LinearRegression())]
)


model4 = lr_pipeline.fit(X_train, y_train)

CV4 = cross_val_score(lr_pipeline, X_train, y_train, cv=5, scoring="neg_mean_squared_error")
cv_rmse4 = np.sqrt(-CV4)

print('Model 4 CV RMSE:', cv_rmse4.mean().round(2))

Model 4 CV RMSE: 85233.37
Model 4 CV RMSE: 85233.37


In [23]:
CV_RMSE_table = {"Models": ["Model 1", "Model 2","Model 3","Model 4"],
         "CV RMSE": [cv_rmse1.mean(),cv_rmse2.mean(),cv_rmse3.mean(),cv_rmse4.mean()]}

pd.DataFrame(CV_RMSE_table)

,Models,CV RMSE
0,Model 1,54922.108540
1,Model 2,53092.364622
2,Model 3,52561.798989
3,Model 4,85233.374661


When doing cross validation we still find that Model 3 has the lowest RMSE, making it the best predictive model.

# Part 3

In [46]:
ct = ColumnTransformer(
  [
    ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"]),
    ("poly_area", PolynomialFeatures(), ["Gr Liv Area"]),
    ("poly_room", PolynomialFeatures(), ["TotRms AbvGrd"])
    

  ],
  remainder = "drop"
)

poly_pipeline = Pipeline(
  [("preprocessing", ct),
  ("regression", LinearRegression())]
).set_output(transform="pandas")

degrees = {'preprocessing__poly_area__degree': np.arange(1, 11),
           'preprocessing__poly_room__degree': np.arange(1, 11)}

grid = GridSearchCV(poly_pipeline, degrees, cv = 5, scoring='r2')
fitted_grid = grid.fit(X, y)

In [47]:
results = grid.cv_results_

results_table = pd.DataFrame({
    "degree for area": results['param_preprocessing__poly_area__degree'],
    "degree for room": results['param_preprocessing__poly_room__degree'],
    "Average R^2": results['mean_test_score']
})

results_table.loc[results_table['Average R^2'].idxmax()]

degree for area           3
degree for room           1
Average R^2        0.557641
Name: 20, dtype: object

The model that has the highest average R^2 after cross validation was the model with a degree of 3 for area and a degree of 1 for room. This is the model that performs the best when using R^2.

The main issue that I find with having this many models is that is lowers the speed of computation. It may be better to use a smaller set of models in order to increase computational speed. There also is a chance that some models may not actually be better than others, but simply have a higher R^2. Sometimes inclued many variables into a model increases the likelihood of overfitting and may result in an artifically high R^2.